# Imports

In [ ]:
import os
import selenium.webdriver as webdriver
from selenium.webdriver.edge.service import Service 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import datetime

# Parameters

In [ ]:
betfair_url = 'https://www.betfair.com/exchange/plus/pt/futebol-apostas-1/today' #USE THIS FOR CAPTURING TODAYS'S GAMES
#betfair_url = 'https://www.betfair.com/exchange/plus/pt/futebol-apostas-1/tomorrow' #USE THIS FOR CAPTURING TOMORROW'S GAMES

betfair_email = '' #FILL WITH YOUR CREDENTIALS
betfair_password = '' #FILL WITH YOUR CREDENTIALS
omqb_email = betfair_email #FILL WITH YOUR CREDENTIALS
omqb_password = betfair_password #FILL WITH YOUR CREDENTIALS

base_xpath = '//*[@id="main-wrapper"]/div/div[2]/div/ui-view/ui-view/div/div/div/div/div[1]/div/div[1]/bf-super-coupon/main/ng-include[3]/section'

#COLUMN HEADERS FOR EACH ODD MARKET
finalresultoddsheaders = ["BH", "LH", "BD", "LD", "BA", "LA"]
bttsoddsheaders = ["BY_BTTS", "LY_BTTS", "BN_BTTS", "LN_BTTS"]
overunder15oddsheaders = ["B_UNDER1_5", "L_UNDER1_5", "B_OVER1_5", "L_OVER1_5"]
overunder25oddsheaders = ["B_UNDER2_5", "L_UNDER2_5", "B_OVER2_5", "L_OVER2_5"]
htoddsheaders = ["BH_HT", "LH_HT", "BD_HT", "LD_HT", "BA_HT", "LA_HT"]

#MARKETS TO CAPTURE ODDS FROM
options = ['Resultado Final', 'Intervalo', 'Ambas as Equipes Marcam', 'Mais/Menos de 1.5 gols',  'Mais/Menos de 2.5 gols']

run_id = "3" #JOB RUN IDENTIFIER
day_of_games = "2023-12-19" #FILL WITH DAY OF GAMES BEING CAPTURED (TODAY OR TOMORROW)
output_file = day_of_games + '_dados_betfair_' + run_id + '.xlsx' #FILENAME FOR BETFAIR DATA
model_file = day_of_games + '_dados_omqb_' + run_id + '.xlsx' #FILENAME FOR BETFAIR + OMQB DATA

# Functions

In [ ]:
#FUNCTION TO CHECK IF ODDS SPREAD IS BIGGER THEN THRESHOLD
def odds_check_spread(row):
    try:
        var_1 = row['LH'] / row['BH'] - 1
    except: 
        var_1 = 1.0
    try:
        var_2 = row['LD'] / row['BD'] - 1
    except: 
        var_1 = 1.0
    try:
        var_3 = row['LA'] / row['BA'] - 1
    except: 
        var_1 = 1.0
        
    #CHANGE HERE IF YOU WANT TO ALLOW SPREADS BIGGER THAN 20%
    if (var_1 > 0.2) or (var_2 > 0.2) or (var_3 > 0.2):
        return 1
    elif (var_1 < 0) or (var_2 < 0) or (var_3 < 0):
        return 1
    else:
        return 0
    
#FUNCTION TO FILL WITH 0.0 WHEN ODD IS NULL/BLANK
def solve_null_odds(array_odds, headers, row_dict):
    for j in range(0,len(array_odds)):
        try:
            row_dict[headers[j]] = float(array_odds[j])
        except:
            row_dict[headers[j]] = 0.0
    return row

# Main Code

### Opening Betfair

In [ ]:
#GET EDGE WEBDRIVER
edge_driver_path = os.path.join(os.getcwd(), 'msedgedriver.exe')
edge_service = Service(edge_driver_path)

#OPEN BROWSER SESSION
browser = webdriver.Edge(service = edge_service)
browser.get(betfair_url) 
browser.maximize_window()

#WAIT FOR COOKIES POPUP
accepted = 0
while accepted == 0:
    try:
        AcceptCookiesButton = browser.find_element(By.ID, 'onetrust-accept-btn-handler')
        AcceptCookiesButton.click()
        accepted = 1
        print("Cookies accepted!")
    except:
        print("Waiting for cookies...")
        time.sleep(2) 
        
#LOGGING IN
BetfairEmailInput = browser.find_element(By.ID, 'ssc-liu')
BetfairEmailInput.clear()
BetfairEmailInput.send_keys(betfair_email)
BetfairPasswordInput = browser.find_element(By.ID, 'ssc-lipw')
BetfairPasswordInput.clear()
BetfairPasswordInput.send_keys(betfair_password)
BetfairLoginButton = browser.find_element(By.ID, 'ssc-lis')
BetfairLoginButton.click()
print("Successfully logged in!")
time.sleep(10)

### Looping through all the games

In [ ]:
#INIT ARRAY FOR EACH ODD MARKET YOU WANT TO CAPTURE
finalresult = []
halftime = []
btts = []
over15 = []
over25 = []

#GET CURRENT TIMESTAMP
timestamp = datetime.datetime.now()
timestampstr = str(timestamp)

#ENTER LOOP FOR EACH ODD MARKET
for i in range(0, len(options)):
    #IF FIRST ODD MARKET (Resultado Final)
    if i == 0:
        print("Capturando ", options[i])
        time.sleep(2)
    #ELSE GO BACK TO FIRST PAGE, THEN SWITCH MARKETS IN DROPDOWN MENU
    else:
        StartButton = browser.find_element(By.XPATH, '//*[@id="main-wrapper"]/div/div[2]/div/ui-view/ui-view/div/div/div/div/div[1]/div/div[1]/bf-super-coupon/main/ng-include[4]/bf-coupon-page-navigation/ul/li[1]/a/label')
        StartButton.click()
        time.sleep(2)
        ToggleElement = browser.find_element(By.XPATH, '//*[@id="main-wrapper"]/div/div[2]/div/ui-view/ui-view/div/div/div/div/div[1]/div/div[1]/bf-super-coupon/main/ng-include[2]/section/div[2]/bf-select/div/label')
        ToggleElement.click()
        time.sleep(2)
        DropElement = browser.find_element(By.XPATH, f'//*[@title="{options[i]}"]')
        DropElement.click()
        print("Capturando ", options[i])
        time.sleep(2)
        
    #SET FIRST PAGE
    pages_to_loop = True
    current_page = browser.current_url
    
    #ENTER LOOP FOR EACH PAGE
    while pages_to_loop:
        print(f'Buscando dados da página {current_page}')
        time.sleep(2)
        
        #ENTER LOOP FOR EACH LEAGUE IN PAGE
        leagues_left = True
        league_counter = 1
        while leagues_left:
            
            try:
                LigaElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[1]/div/span')
                print(f'Capturando {LigaElement.text}')
                
                for div in ["[1]", "[2]"]:
                    #ENTER LOOP FOR EACH GAME IN LEAGUE
                    games_left = True
                    games_counter = 1
                    while games_left:
                        row = {}
                        try:
                            row["RunTimestamp"] = timestampstr
                            row["RunID"] = run_id
                        
                            #ATTEMPT TO GET DATE
                            try:
                                row["GameTimestamp"] = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[1]/a/event-line/section/bf-livescores/section/div/div/data-bf-livescores-start-date/ng-include/div/div').text
                            #OTHERWISE RETURN LIVE
                            except:
                                row["GameTimestamp"] = "Ao Vivo"
         
                            #GET TEAMS AND ODDS
                            row["HomeTeam"] = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[1]/a/event-line/section/ul[1]/li[1]').text
                            row["AwayTeam"] = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[1]/a/event-line/section/ul[1]/li[2]').text
                            row["Liga"] = LigaElement.text
                            
                            #IF MARKET HAS 3 PAIRS OF ODDS
                            if options[i] in ['Resultado Final', 'Intervalo']:
                                BackDiv1Element = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[1]/ours-price-button[1]/button/label[1]').text
                                LayDiv1Element = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[1]/ours-price-button[2]/button/label[1]').text
                                BackDiv2Element = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[2]/ours-price-button[1]/button/label[1]').text
                                LayDiv2Element = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[2]/ours-price-button[2]/button/label[1]').text
                                BackDiv3Element = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[3]/ours-price-button[1]/button/label[1]').text
                                LayDiv3Element = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[3]/ours-price-button[2]/button/label[1]').text
                            #IF MARKET HAS 2 PAIRS OF ODDS
                            else:
                                BackDiv1Element = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[1]/ours-price-button[1]/button/label[1]').text
                                LayDiv1Element = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[1]/ours-price-button[2]/button/label[1]').text
                                BackDiv2Element = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[2]/ours-price-button[1]/button/label[1]').text
                                LayDiv2Element = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table{div}/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[2]/ours-price-button[2]/button/label[1]').text
                            
                            #FOR EACH MARKET APPEND ROW IN CORRECT ARRAY
                            if options[i] == 'Resultado Final':
                                oddstext = [BackDiv1Element, LayDiv1Element, BackDiv2Element, LayDiv2Element, BackDiv3Element, LayDiv3Element]
                                finalresult.append(solve_null_odds(oddstext, finalresultoddsheaders, row))
                                print(row)
                            elif options[i] == 'Intervalo':
                                oddstext = [BackDiv1Element, LayDiv1Element, BackDiv2Element, LayDiv2Element, BackDiv3Element, LayDiv3Element]
                                halftime.append(solve_null_odds(oddstext, htoddsheaders, row))
                                print(row)
                            elif options[i] == 'Ambas as Equipes Marcam':
                                oddstext = [BackDiv1Element, LayDiv1Element, BackDiv2Element, LayDiv2Element]
                                btts.append(solve_null_odds(oddstext, bttsoddsheaders, row))
                                print(row)
                            elif options[i] == 'Mais/Menos de 1.5 gols':
                                oddstext = [BackDiv1Element, LayDiv1Element, BackDiv2Element, LayDiv2Element]
                                over15.append(solve_null_odds(oddstext, overunder15oddsheaders, row))
                                print(row)
                            elif options[i] == 'Mais/Menos de 2.5 gols':
                                oddstext = [BackDiv1Element, LayDiv1Element, BackDiv2Element, LayDiv2Element]
                                over25.append(solve_null_odds(oddstext, overunder25oddsheaders, row))
                                print(row)
                            
                            games_counter += 1
                            time.sleep(0.5)
                            
                        #EXIT GAME LOOP
                        except:
                            print("Essa liga não tem mais jogos!")
                            games_left = False
                            time.sleep(0.5)
                    
                league_counter += 1
                time.sleep(0.5)
                
            #EXIT LEAGUE LOOP
            except:
                print("Não há mais ligas nessa página!")
                leagues_left = False
                time.sleep(2)
        
        #MOVE TO NEXT PAGE
        previous_page = current_page
        try:
            NextButton = browser.find_element(By.XPATH, '//*[@id="main-wrapper"]/div/div[2]/div/ui-view/ui-view/div/div/div/div/div[1]/div/div[1]/bf-super-coupon/main/ng-include[4]/bf-coupon-page-navigation/ul/li[4]/a/label')
            NextButton.click()
        except:
            pages_to_loop = False
        current_page = browser.current_url
        
        #IF LAST PAGE, END LOOP
        if previous_page == current_page:
            pages_to_loop = False
        time.sleep(2)

### Creating Betfair data dataframe

In [ ]:
#CHECK LEN OF EACH ARRAY 
#OBS: HT, BTTS, Over Goals CAN HAVE LESS ROWS IF YOU CAPTURED LIVE GAMES
print(len(finalresult))
print(len(halftime))
print(len(btts))
print(len(over15))
print(len(over25))

In [ ]:
#CREATING DATAFRAMES FOR EACH MARKET
df_finalresult = pd.DataFrame.from_dict(finalresult)
df_finalresult['Dia'] = day_of_games
df_finalresult['ID'] = df_finalresult['HomeTeam'] + ' vs ' + df_finalresult['AwayTeam'] + ' ' + df_finalresult['Dia']
df_finalresult['Spread'] = df_finalresult.apply(odds_check_spread, axis=1)
df_finalresult = df_finalresult.set_index('ID')

df_halftime = pd.DataFrame.from_dict(halftime)
df_halftime['Dia'] = day_of_games
df_halftime['ID'] = df_halftime['HomeTeam'] + ' vs ' + df_halftime['AwayTeam'] + ' ' + df_halftime['Dia']
df_halftime = df_halftime.set_index('ID')
df_halftime = df_halftime.drop(columns=['RunTimestamp','RunID','GameTimestamp','HomeTeam','AwayTeam','Liga','Dia'])

df_btts = pd.DataFrame.from_dict(btts)
df_btts['Dia'] = day_of_games
df_btts['ID'] = df_btts['HomeTeam'] + ' vs ' + df_btts['AwayTeam'] + ' ' + df_btts['Dia']
df_btts = df_btts.set_index('ID')
df_btts = df_btts.drop(columns=['RunTimestamp','RunID','GameTimestamp','HomeTeam','AwayTeam','Liga','Dia'])

df_over15 = pd.DataFrame.from_dict(over15)
df_over15['Dia'] = day_of_games
df_over15['ID'] = df_over15['HomeTeam'] + ' vs ' + df_over15['AwayTeam'] + ' ' + df_over15['Dia']
df_over15 = df_over15.set_index('ID')
df_over15 = df_over15.drop(columns=['RunTimestamp','RunID','GameTimestamp','HomeTeam','AwayTeam','Liga','Dia'])

df_over25 = pd.DataFrame.from_dict(over25)
df_over25['Dia'] = day_of_games
df_over25['ID'] = df_over25['HomeTeam'] + ' vs ' + df_over25['AwayTeam'] + ' ' + df_over25['Dia']
df_over25 = df_over25.set_index('ID')
df_over25 = df_over25.drop(columns=['RunTimestamp','RunID','GameTimestamp','HomeTeam','AwayTeam','Liga','Dia'])

In [ ]:
#JOINING ALL DFS INTO ONE
df = df_finalresult.join(df_halftime, rsuffix = '_HT').join(df_btts, rsuffix = '_BTTS').join(df_over15, rsuffix = '_OVER15').join(df_over25, rsuffix = '_OVER25')
df.head(5)

In [ ]:
#EXPORTING EXCEL FILE
df.to_excel(output_file)
print("Excel exported!")

### Opening OMQB

In [ ]:
#GET OMQB AND LOG IN
browser.get('https://om-qb.com')
EmailinputElement = browser.find_element(By.ID, 'id_login')
EmailinputElement.send_keys(omqb_email)
SenhainputElement = browser.find_element(By.ID, 'id_password')
SenhainputElement.send_keys(omqb_password)
SenhainputElement.send_keys(Keys.ENTER)
print("Successfully logged in!")
time.sleep(5)

#GO TO LAY MODEL
ModeloAbaElement = browser.find_element(By.XPATH, '//*[@id="sidebar"]/ul/li[3]/a')
ModeloAbaElement.click()
LayModelElement = browser.find_element(By.XPATH, '//*[@id="sidebar"]/ul/li[3]/ul/li[1]/a')
LayModelElement.click()
time.sleep(2)

### Compiling Lay Model results

In [ ]:
#GET LAY MODEL VERSION FOR CONTROL
LayVersionElement = browser.find_element(By.XPATH, '/html/body/div[4]/div[1]/div[1]/h1').text
print(LayVersionElement)

#LOOP THROUGH ALL GAMES
for row in range(0, len(df)):
    
    #IF FLAG SPREAD = 1 (ABOVE 20%) PASS MODEL
    if df.iloc[row]['Spread'] == 1:
        df.at[df.index[row],LayVersionElement] = 'Sugestão: Fique de fora no modelo - OMQBI:'
    #ELSE GET MODEL OUTPUT
    else:
        home = str(df.iloc[row]['BH'])
        draw = str(df.iloc[row]['BD'])
        away = str(df.iloc[row]['BA'])
    
        time.sleep(1)
    
        HomeinputElement = browser.find_element(By.NAME, 'home-odds')
        HomeinputElement.send_keys(home)
        DrawinputElement = browser.find_element(By.NAME, 'draw-odds')
        DrawinputElement.send_keys(draw)
        AwayinputElement = browser.find_element(By.NAME, 'away-odds')
        AwayinputElement.send_keys(away)
        AwayinputElement.send_keys(Keys.ENTER)
    
        time.sleep(2)
    
        ResultadoElement = browser.find_element(By.XPATH, '/html/body/div[4]/div[1]/div[2]').text
        print(str(df.index[row]) , ' ', ResultadoElement)

        df.at[df.index[row],LayVersionElement] = ResultadoElement

### Navigating to the Back Model

In [ ]:
#GO TO BACK MODEL
time.sleep(5)
ModeloAbaElement = browser.find_element(By.XPATH, '//*[@id="sidebar"]/ul/li[3]/a')
ModeloAbaElement.click()
BackModelElement = browser.find_element(By.XPATH, '//*[@id="sidebar"]/ul/li[3]/ul/li[2]/a')
BackModelElement.click()
time.sleep(5)

### Compiling Back Model results 

In [ ]:
#GET BACK MODEL VERSION
BackVersionElement = browser.find_element(By.XPATH, '/html/body/div[4]/div[1]/div[1]/h1').text
print(BackVersionElement)

for row in range(0, len(df)):
    
    #IF FLAG SPREAD = 1 (ABOVE 20%) PASS MODEL
    if df.iloc[row]['Spread'] == 1:
        df.at[df.index[row],BackVersionElement] = 'Sugestão: Fique de fora no modelo - OMQBI:'
    #ELSE GET MODEL OUTPUT
    else:
        home = str(df.iloc[row]['BH'])
        draw = str(df.iloc[row]['BD'])
        away = str(df.iloc[row]['BA'])
    
        time.sleep(1)
    
        HomeinputElement = browser.find_element(By.NAME, 'home-odds')
        HomeinputElement.send_keys(home)
        DrawinputElement = browser.find_element(By.NAME, 'draw-odds')
        DrawinputElement.send_keys(draw)
        AwayinputElement = browser.find_element(By.NAME, 'away-odds')
        AwayinputElement.send_keys(away)
        AwayinputElement.send_keys(Keys.ENTER)
    
        time.sleep(2)
    
        ResultadoElement = browser.find_element(By.XPATH, '/html/body/div[4]/div[1]/div[2]').text
        print(str(df.index[row]) , ' ', ResultadoElement)

        df.at[df.index[row],BackVersionElement] = ResultadoElement

### Saving Model output Excel

In [ ]:
df.to_excel(model_file)
print("Excel exported!")

### Closing Edge Webdriver

In [ ]:
browser.close()
browser.quit()
print("Browser closed!")